# Understanding the Kaggle Data
The dataset acquired from [Kaggle](https://www.kaggle.com/code/martinkk5575/language-detection/data) contains words from several different languages. The noise contained in the dataset are duplicate words. To reduce this noise, the words will be broken down into single and double characters, then rated based on how often they show up in that respective language.

In [2]:
import pandas as pd
import numpy as np

# Import data*
fileName = "dataset.csv"
data = pd.read_csv(fileName)

data

,Text,language
0,klement gottwaldi surnukeha palsameeriti ning ...,Estonian
1,sebes joseph pereira thomas på eng the jesuit...,Swedish
2,ถนนเจริญกรุง อักษรโรมัน thanon charoen krung เ...,Thai
3,விசாகப்பட்டினம் தமிழ்ச்சங்கத்தை இந்துப் பத்திர...,Tamil
4,de spons behoort tot het geslacht haliclona en...,Dutch
...,...,...
21995,hors du terrain les années et sont des année...,French
21996,ใน พศ หลักจากที่เสด็จประพาสแหลมมลายู ชวา อินเ...,Thai
21997,con motivo de la celebración del septuagésimoq...,Spanish
21998,年月，當時還只有歲的她在美國出道，以mai-k名義推出首張英文《baby i like》，由...,Chinese


In [3]:
from sklearn.model_selection import train_test_split

X=data['Text'] # Feature matrix
y=data['language'] # Label

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Save the languages into a DataFrame that we aren't modifying
language_list = set(y)

In [4]:
X_train

5207     สัมประสิทธิ์ฮอลล์ ฟิสิกส์ไฟฟ้า เกี่ยวกับสนามแม...
4450     เกิดวันที่  พฤศจิกายน ภาคอะนิเมะ คดีฆาตกรรมบนจ...
7033     i omgivningarna runt manigotagan river park re...
487      நிஞ்சா ஹட்டோரி 忍者ハットリくん ninja hattori என்பது க...
19537    эта страница деятельности м в ломоносова — ярк...
                               ...                        
11964    باباجان غفورف تاریخ‌دان و نویسندهٔ کتاب تاریخ ...
21575    en  fue invitado por fernando ii para ocupar l...
5390     doğu kanada atabasklarına geleneksel olarak dü...
860      پژواک د يوې ځانگړې پروژې په توگه د اساسي قانون...
15795    テンサイについては糖分を高度に精製する必要があることからサトウキビと同じような黒糖を作るのは...
Name: Text, Length: 17600, dtype: object

In [5]:
def feature_engineering(dataframe, chars):
    arr = dataframe.to_numpy()
    new_arr = np.zeros((len(arr), len(chars)))
    i=0
    j=0
    for text in arr:
        sentence = text
        count = 0
        j=0
        for char in chars:
            
            for letter in sentence:
                if letter == char:
                    count = count + 1
                fraction = count/len(sentence)
            new_arr[i,j] = fraction
            j = j + 1
        
        i = i + 1
            
    return new_arr

In [6]:
def numpy_into_dataframe(array, chars):
    data_frame = pd.DataFrame(array, columns = chars)
    return data_frame

In [7]:
panda = X_train.head()
panda

5207     สัมประสิทธิ์ฮอลล์ ฟิสิกส์ไฟฟ้า เกี่ยวกับสนามแม...
4450     เกิดวันที่  พฤศจิกายน ภาคอะนิเมะ คดีฆาตกรรมบนจ...
7033     i omgivningarna runt manigotagan river park re...
487      நிஞ்சா ஹட்டோரி 忍者ハットリくん ninja hattori என்பது க...
19537    эта страница деятельности м в ломоносова — ярк...
Name: Text, dtype: object

In [8]:
first_22_features = ['e', 't', 'a', 'i', 'o', 'á', 'é', 'í']

test_data = feature_engineering(panda, first_22_features)
test_data_2 = numpy_into_dataframe(test_data, first_22_features)

In [9]:
test_data_2

,e,t,a,i,o,á,é,í
0,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,0.106299,0.15748,0.244094,0.295276,0.318898,0.318898,0.318898,0.318898
3,0.000000,0.00692,0.013841,0.020761,0.024221,0.024221,0.024221,0.024221
4,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


## Creating Features: Most Used Characters
To train the models to determine which language is being used by the user, we first need to know which characters are used in each language. The best approach for this is to used the dataset, _which is already using the characters from each language_, and find the **most used characters** in them.

### Kaggle Resources
----
The following code is taken from the [Kaggle](https://www.kaggle.com/code/martinkk5575/language-detection/notebook) to understand how to process characters for each language. Kaggle uses the `CountVectorizor` Method from the `sklearn` module to tokenize the characters into readable 1's and 0's. Then, it counts how many times that character has been used in the sample data provided.

This method reduces the necessity of locating alphabets for each language and creating custom functions to find the most used characters in the dataset.
 
**To summarize Kaggle's findings: Languages based off the Latin Alphabet are easier to differentiate from each other in the data set while Languages with their own Alphabet, _like Chinese and Japanese_, can be differentiated by single characters alone.**


This code here fits and transforms the X_train and X_test data sets into readable 1's and 0's and counts the number of times a specific character shows up in the datasets.The `min_df` parameter tells `CountVectorizor` to save any characters that are used **at least 1% of the time** in the dataset.

These matrices are saved as `X_top1Percent_train_raw` and `X_top1Percent_test_raw`.

In [27]:
# This will create a list of single and double characters from the top 1% of to be used as features
from sklearn.feature_extraction.text import CountVectorizer

top1PrecentMixtureVectorizer = CountVectorizer(analyzer='char', ngram_range=(1,2), min_df=1e-2)

X_top1Percent_train_raw = top1PrecentMixtureVectorizer.fit_transform(X_train)
X_top1Percent_test_raw = top1PrecentMixtureVectorizer.transform(X_test)

The `train_lang_dict()` function takes in the raw vectorized X_train and y_train data sets and converts them into readable dictionaries.

In [13]:
# This Command from Kaggle connects the character features to their specific language

# Aggregate Unigrams per language
def train_lang_dict(X_raw_counts, y_train):
    lang_dict = {}
    for i in range(len(y_train)):
        lang = y_train[i]
        v = np.array(X_raw_counts[i])
        if not lang in lang_dict:
            lang_dict[lang] = v
        else:
            lang_dict[lang] += v
            
    # to relative
    for lang in lang_dict:
        v = lang_dict[lang]
        lang_dict[lang] = v / np.sum(v)
        
    return lang_dict

In [28]:
language_dict_top1Percent = train_lang_dict(X_top1Percent_train_raw.toarray(), y_train.values)

top1PercentFeatures = top1PrecentMixtureVectorizer.get_feature_names_out()

The `getRelevantGramsPerLanguage()` function processes the dictionary and returns a dictionary with only the top 50 **most used** characters for **each** language. This number _can_ be changed by setting `top=x` when you call `getRelevantGramsPerLanguage()`

In [34]:
def getRelevantGramsPerLanguage(features, language_dict, top=50):
    relevantGramsPerLanguage = {}
    for lang in language_list:
        chars = []
        relevantGramsPerLanguage[lang] = chars
        v = language_dict[lang]
        sortIndex = (-v).argsort()[:top]
        for i in range(len(sortIndex)):
            chars.append(features[sortIndex[i]])
    return relevantGramsPerLanguage

Below Displays the top 8 and 10 Characters from each language in a DataFrame 

In [35]:
top8PerLanguage_dict = getRelevantGramsPerLanguage(top1PercentFeatures, language_dict_top1Percent, top=8)
top8PerLanguage_dict

{'Japanese': ['の', '、', 'に', 'た', 'る', '。', 'は', 'と'],
 'Indonesian': ['a', ' ', 'n', 'e', 'i', 'r', 'u', 't'],
 'Latin': [' ', 'i', 'a', 'e', 't', 's', 'n', 'r'],
 'Estonian': [' ', 'a', 'i', 'e', 's', 't', 'l', 'n'],
 'Thai': ['า', 'น', 'ร', ' ', 'ก', 'อ', '่', 'เ'],
 'French': [' ', 'e', 'a', 'n', 's', 'i', 'r', 't'],
 'Russian': [' ', 'о', 'и', 'е', 'а', 'н', 'с', 'р'],
 'Swedish': [' ', 'e', 'r', 'a', 'n', 't', 'i', 's'],
 'Chinese': ['，', '的', '。', ' ', '年', '在', '、', '一'],
 'Portugese': [' ', 'a', 'e', 'o', 's', 'i', 'r', 'd'],
 'Romanian': [' ', 'e', 'a', 'i', 'r', 'n', 't', 'u'],
 'Persian': [' ', 'ا', 'ی', 'ر', 'د', 'ن', 'ه', 'و'],
 'Arabic': [' ', 'ا', 'ل', 'ي', 'ال', 'م', 'و', ' ا'],
 'Tamil': ['்', ' ', 'க', 'ு', 'ி', 'த', '் ', 'ப'],
 'Pushto': [' ', 'و', 'ا', 'ه', 'ي', 'ه ', 'د', 'ر'],
 'Dutch': [' ', 'e', 'n', 'a', 'i', 'r', 't', 'o'],
 'Turkish': [' ', 'a', 'e', 'i', 'n', 'r', 'l', 'ı'],
 'Korean': [' ', '이', '의', '다', '의 ', '에', '는', '는 '],
 'Spanish': [' ', 'e', 'a',

In [66]:
top10PerLanguage_dict = getRelevantGramsPerLanguage(top1PercentFeatures, language_dict_top1Percent, top=10)
top10PerLanguage_dict

{'Japanese': ['の', '、', 'に', 'た', 'る', '。', 'は', 'と', 'ー', 'を'],
 'Indonesian': ['a', ' ', 'n', 'e', 'i', 'r', 'u', 't', 's', 'an'],
 'Latin': [' ', 'i', 'a', 'e', 't', 's', 'n', 'r', 'u', 'o'],
 'Estonian': [' ', 'a', 'i', 'e', 's', 't', 'l', 'n', 'u', 'o'],
 'Thai': ['า', 'น', 'ร', ' ', 'ก', 'อ', '่', 'เ', 'ง', 'ม'],
 'French': [' ', 'e', 'a', 'n', 's', 'i', 'r', 't', 'l', 'e '],
 'Russian': [' ', 'о', 'и', 'е', 'а', 'н', 'с', 'р', 'т', 'в'],
 'Swedish': [' ', 'e', 'r', 'a', 'n', 't', 'i', 's', 'd', 'l'],
 'Chinese': ['，', '的', '。', ' ', '年', '在', '、', '一', '中', 'a'],
 'Portugese': [' ', 'a', 'e', 'o', 's', 'i', 'r', 'd', 'n', 't'],
 'Romanian': [' ', 'e', 'a', 'i', 'r', 'n', 't', 'u', 'l', 'o'],
 'Persian': [' ', 'ا', 'ی', 'ر', 'د', 'ن', 'ه', 'و', 'م', 'ت'],
 'Arabic': [' ', 'ا', 'ل', 'ي', 'ال', 'م', 'و', ' ا', 'ن', 'ت'],
 'Tamil': ['்', ' ', 'க', 'ு', 'ி', 'த', '் ', 'ப', 'ம', 'ட'],
 'Pushto': [' ', 'و', 'ا', 'ه', 'ي', 'ه ', 'د', 'ر', 'ل', 'ن'],
 'Dutch': [' ', 'e', 'n', 'a', 'i', 

In [60]:
def dictToArray(dict, languages=language_list):
    '''Converts a Language dictionary to an array and removes the duplicate values'''
    char = []

    for lang in languages:
        arr = dict[lang]
        char = char + arr
    
    dict_array = list(set(char))

    return dict_array

In [69]:
dict8_array = dictToArray(top8PerLanguage_dict)

print(dict8_array)
print("Total Number of Characters:", len(dict8_array))

['t', 'ه ', '் ', 'l', 'р', 'م', 'e', 'น', 'ி', 'ப', 'の', 'n', 'ل', 'は', ' ', 'н', 'ن', '年', 'த', 'u', 'ı', 'と', 'о', '이', '는', 'ก', '்', 'க', 'd', 'า', 'د', 'o', '्', '의', 'е', 'ร', 'и', '在', ' ا', '는 ', 'स', '의 ', 'ي', 'े', 's', 'а', '่', 'с', '、', 'ه', '。', 'ر', '다', 'に', 'و', 'i', 'ு', '에', 'र', 'ک', '，', '一', 'ی', 'ि', 'r', 'ा', 'た', '的', 'เ', 'क', 'ال', 'อ', 'ا', 'る', 'a']
Total Number of Characters: 75


In [70]:
dict10_array = dictToArray(top10PerLanguage_dict)

print(dict10_array)
print("Total Number of Characters:", len(dict10_array))

['an', 'ி', 'ப', 'ل', 'は', 'ن', 'த', '年', '는', 'd', 'า', 'د', 'स', 'े', 'ு', 'h', 'r', 'た', 'เ', '을', 'อ', 'a', 'م', 'ت', 'ı', ' ', 'н', '이', 'u', 'と', 'о', 'ก', '்', 'க', '의', 'е', '의 ', 's', '่', 'ه', 'र', '。', '다', 'に', 'و', '에', 'ง', '，', '一', 'ि', 'ा', '하', 'k', ' क', 'ا', 'ー', 't', 'ه ', 'р', 'e', 'の', 'e ', 'n', '는 ', '्', 'ร', 'を', '在', 'क', 'ر', 'ک', 'ม', 'ہ', 'ے', 'т', '中', '் ', 'l', 'น', 'न', 'o', 'и', ' ا', 'ي', 'а', 'в', 'с', '、', 'i', 'ی', 'ம', '的', 'ட', 'ال', 'る']
Total Number of Characters: 95


## Training the Model: RandomForests
Now that we know the top 8 and 10 used characters for each language, we can use them to train our model.

First, we'll create a `CountVectorizor` instance to read in the dictionaries.

In [71]:
dictVectorizor = CountVectorizer(analyzer='char', ngram_range=(1,2))

X_top8 = dictVectorizor.fit_transform(dict8_array)

In [72]:
from sklearn.ensemble import RandomForestClassifier

rfc = RandomForestClassifier()

rfc.fit(X_top8, y_test)

ValueError: Found input variables with inconsistent numbers of samples: [75, 4400]

In [ ]:
y_pred = rfc.predict(X_top1Percent_test_raw)

In [ ]:
from sklearn.metrics import accuracy_score

print("Accuracy Score of top 1 percent used characters:",accuracy_score(y_test, y_pred))

Accuracy Score of top 1 percent used characters: 0.9802272727272727


In [ ]:
# Create these arrays into dictonaries
english_alpha = [a, b, c, d, e, f, g, h, i, j, k, l, m, n, o, p, q, r, s, t, u, v, w, x, y, z]
estonian_alpha = [A, B, D, E, F, G, H, I, J, K, L, M, N, O, P, R, S, Š, Z, Ž, T, U, V, Õ, Ä, Ö, Ü]
swedish_alpha = [a, b, c, d, e, f, g, h, i, j, k, l, m, n, o, p, q, r, s, t, u, v, w, x, y, z, å, ä, ö]
thai_alpha = [ก, ข, ค, ฅ, ฆ, ง, จ, ฉ, ช, ฌ, ญ, ฎ, ฏ, ฐ, ฑ, ฒ, ณ, ด, ต, ถ, ท, ธ, น, บ, บ, ผ, ฝ, พ, ฟ, ภ, 
               ม, ย, ร, ล, ว, ศ, ษ, ส, ห, ฬ, อ, ฮ] 
tamil_alpha = [அ, ஆ, இ, ஈ, உ, ஊ, எ, ஏ, ஐ, ஒ, ஓ, ஔ, க, ங, ச, ஞ, ட, ண, த, ந, ன, ப, ம, ய, ர, ற, ல, ள, ழ, வ]
dutch_alpha = english_alpha
japanese_alpha = [ぁ, あ, ぃ, い, ぅ, う, ぇ, え, ぉ, お, か, が, き, ぎ, く, ぐ, け, げ, こ, ご, さ, ざ, し, じ, す, ず,
                  せ, ぜ, そ, ぞ, た, だ, ち, ぢ, っ, つ, づ, て, で, と, ど, な, に, ぬ, ね, の, は, ば, ぱ, ひ, び, ぴ,
                  ふ, ぶ, ぷ, へ, べ, れ, る, り, ら, よ, ょ, ゆ, ゅ, や, ゃ, も, め, む, み, ま, ぽ, ぼ, ほ, ぺ, ろ, ゎ,
                  わ, ゐ, ゑ, を, ん, ゔ, ゕ, ゖ,  ゚, ゛, ゜, ゝ, ゞ, ゟ, ゠, ァ, ア, サ, ゴ, コ, ゲ, ケ, グ, ク, ギ, キ,
                  ガ, カ, オ, ォ, エ, ェ, ウ, ゥ, イ, ィ, ザ, シ, ジ, ス, ズ, セ, ゼ, ソ, ゾ, タ ,ダ ,チ ,ヂ, ッ, ツ, ヅ,
                  テ, デ, ト, ホ, ペ, ベ, ヘ, プ, ブ, フ, ピ, ビ, ヒ, パ, バ, ハ, ノ, ネ, ヌ, ニ, ナ, ド, ボ, ポ, マ, ミ, 
                  ム, メ, モ, ャ, ヤ, ュ, ユ, ョ, ヨ, ラ, リ, ル, レ, ロ, ヮ, ㍿, ㍐, ヿ, ヾ, ヽ, ー, ・, ヺ, ヹ, ヸ, ヷ,
                  ヶ, ヵ, ヴ, ン, ヲ, ヱ, ヰ, ワ]
turkish_alpha = [a, b, c, d, e, f, g, h, i, j, k, l, m, n, o, p, r, s, t, u, v, y, z, ç, ğ, ı, İ, î, ö, ş, ü]
latin_alpha = english_alpha
urdu_alpha = [ش,س,ژ,ز,ڑ,ر,ذ,ڈ,د,خ,ح,چ,
              ج,ث,ٹ,ت,پ,ب,آ,ا,ے,ی,ھ,ہ,و,ں,ن,م,ل,گ,ک,ق,ف,غ,ع,ظ,ط,ض,ص]
indonesian_alpha = english_alpha
portuguese_alpha = [ç, á, é, í, ó, ú, â, ê, ô, ã, õ, à, è, ì, ò, ù, a, b, c, d, e, f, g, h, i, j, k, l, m, n, o, p, q, r, s, t, u, v, w, x, y, z]
french_alpha = [ç, é, â, ê, î, ô, û, à, è, ì, ò, ù, ë, ï, ü, a, b, c, d, e, f, g, h, i, j, k, l, m, n, o, p, q, r, s, t, u, v, w, x, y, z]
chinese_alpha = [胡, 赛, 尼, 本, 人, 和, 小, 说, 的, 主, 人, 公, 阿, 米, 尔, 一, 样, 都, 是, 出, 生, 在, 阿, 富, 汗, 首, 都, 
                 喀, 布, 尔, 少, 年, 时, 代, 便, 离, 开, 了, 这, 个, 国, 家, 。, 胡, 赛, 尼, 直, 到, 年, 小, 说, 出, 版, 之, 
                 后, 才, 首, 次, 回, 到, 已, 经, 离, 开, 年, 的, 祖, 国, 。, 他, 在, 苏, 联, 入, 侵, 时, 离, 开, 了, 阿, 富, 
                 汗, 而, 他, 的, 很, 多, 童, 年, 好, 友, 在, 阿, 富, 汗, 生, 活, 在, 他, 们, 出, 发, 之, 前, 罗, 伯, 特, 伊,
                 达, 尔, 文, 卷, 查, 尔, 斯, 赖, 尔, 所, 著, 地, 质, 学, 原, 理, 在, 南, 美, 他, 得, 到, 第, 卷, 该, 书, 将, 
                 地, 形, 地, 貌, 解, 释, 为, 漫, 长, 历, 史, 时, 间, 渐, 进, 演, 变, 的, 的, 结, 果, 当, 他, 旅, 程, 的, 第, 
                 站, 抵, 达, 圣, 地, 亚, 哥, 佛, 得, 角, 的, 时, 候, 达, 尔, 文]
korean_alpha = [ᄁ,ᄂ,ᄃ,ᄄ,ᄅᄆᄇ,ᄈ,ᄉ,ᄊ,ᄋ,ᄌᄍ,ᄎ,ᄏ,ᄐ,ᄑᄒ,아,악,안,알,암,압,앙,앞애,액,앵야,얀,약,양,얘,어,억,
                언,얼,엄,업,엉,에,여,역,연,열,염,엽,영,예,ᄀ,여,역,연,열,염,엽,영,예,오,옥,온,올,옴,옹,와,완,왈,왕,왜,외,왼,
                요,욕,용,우,욱,운,울,움,웅,워,원,월,위,유,육,윤,율,융,윷,으,은,을,음읍,응,의,이,익,인,일,임,입,잉,잎]
hindi_alpha = [ऄ, अ, आ, इ, ई, उ, ऊ, ऋ, ऌ, ऍ, ऎ, ए, ऐ, ऑ, ऒ, ओ, औ, क, ख, ग, घ, ङ, च, छ, ज, झ, प, ऩ, न, ध, द, 
               थ, त, ण, ढ, ड, ठ, ट, ञ, फ, ब, भ, म, य, र, ऱ, ल, ळ, ऴ, व, श, ष, ४, ३, २, १, ०, ॥, ।, ॡ, ॠ, ॐ, ऽ, 
               ह, स, ५, ६, ७, ॲ, ॳ, ॴ, ॵ, ॶ, ॷ, ॹ, ॺ, ॻ, ॼ, ॾ, ॿ, ೱऀँं, ः, ऺ, ऻ, ा, ि, ी, ॎ, ॏॕैेॣॢ, ॗ]
spanish_alpha = [á, é, í, ó, ú, ñ, ü, a, b, c, d, e, f, g, h, i, j, k, l, m, n, o, p, q, r, s, t, u, v, w, x, y, z]
pushto_alpha = [,ﺏ ,پ ,ﺕ ,ټ ,ﺙ ,ﺝ ,چ ,ﺡ ,ﺥ ,څ ,ځ ,ﺩ ,ډ ,ﺫ ,ﺭ ,ړ ,ﺯ ,ژ ,ږ ,ﺱ ,ﺵ ,ښ ,ﺹ ,ﺽ ,ﻁ ,ﻅ ,ﻉ ,ﻍ ,ﻑ ,ﻕ ,ک ,ګ ,ﻝ ,ﻡ ,ﻥ ,ڼ, ,ﻭ ,ه ,ۀ ,ي ,ې ,ی ,ۍ ,ئ]
persian_alpha = [,ش,س,ژ,ز,ر,ذ,د,خ,ح,چ,ج,ث,ت,پ,ب,آ,ا,ص,ض,ط,ظ,ع,غ,ف,ق,ک,گ,ل,م,ن,و,ه,ی]
romanian_alpha = [ă, â, î, ș, ş, ț, ţ, a, b, c, d, e, f, g, h, i, j, k, l, m, n, o, p, q, r, s, t, u, v, w, x, y, z]
russian_alpha = [б, в, г, д, ж, з, к, л, м, н, п, р, с, т, ф, х, ц, ч, ш, щ, а, е, ё, и, о, у, ы, э, ю, я, й]
arabic_alpha = [ش,س,ز,ر,ذ,د,خ,ح,ج,ث,ت,ب,ا,ء,ي,و,ه,ن,م,ل,ك,ق,ف,غ,ع,ظ,ط,ض,ص]


In [ ]:
# Function to calculate Average Word Length of a single string
def avg_word_len(string):
    # Split string up and find total amount of words present
    words = string.split()
    wordCount = len(words)
    
    # Calculate actual average  
    ch = 0
    for word in words:
        ch += len(word) # Add up all chars
    avg = ch / wordCount # Divide sum of chars by amount of words present
    return avg

In [ ]:
# Function to calculate the Average Sentence Length across a piece of text
def avg_sent_len(text):
  sentences = text.split(".") # Split the text into a list of sentences.
  words = text.split(" ") # split the input text into a list of separate words
  if(sentences[len(sentences)-1]==""): # if the last value in sentences is an empty string
    average_sentence_length = len(words) / len(sentences)-1
  else:
    average_sentence_length = len(words) / len(sentences)
  return average_sentence_length # returning avg length of sentence
  
ans = avg_sent_len("I am going.to see you later") # function call
print(ans)

In [ ]:
# Lets test this function (avg_sent_len)          #Mr., Dr., Ms., etc. are words that may be a problem for this function
avg_sent_len(input("Provide a body of text: "))

In [ ]:
# One Hot Encoding                                #Attempt failed lol
from sklearn.preprocessing import OneHotEncoder

cat_encoder = OneHotEncoder() # instantiate one hot encoder
cat_encoder2 = cat_encoder.fit_transform(data_trans)
cat_encoder2 #sparse matrix

cat_encoder2.toarray()

array([[1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 

In [ ]:
# One Hot Encoding
features = pd.get_dummies(data_trans)

In [ ]:
features

,0_ أ,1_ ہ,2_या,3_ د,4_行,5_》,6_こ,7_ng,8_े,9_अ,...,554_ட,555_்க,556_ा,557_서,558_을,559_த்,560_ु,561_t,562_も,563_ं
Chars,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1


In [ ]:
data_trans

,0,1,2,3,4,5,6,7,8,9,...,554,555,556,557,558,559,560,561,562,563
Chars,ed,た。,m,ی,nd,d,と,د,ار,和,...,ใ,ไ,h,d,่,क,อง,‌,有,و
